## Movie Recommendations based on Ratings

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using __Alternating Least Squares (ALS)__. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.


In [41]:
# Initialize pyspark
import findspark
findspark.init()
import pyspark

In [42]:
# Initialize and create ba spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('movie').getOrCreate()

In [43]:
# Using Spark to read the movie lens data set.
data = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [44]:
# Printing the first few rows of the dataframe
data.show(4)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
+-------+------+------+
only showing top 4 rows



In [45]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [46]:
#Since the dataset is smaller, we can use 80% and 20% for train-test splits
train_data,test_data = data.randomSplit([0.8,0.2])

In [47]:
train_data.describe().show()

+-------+-----------------+------------------+------------------+
|summary|          movieId|            rating|            userId|
+-------+-----------------+------------------+------------------+
|  count|             1215|              1215|              1215|
|   mean|49.76131687242798| 1.757201646090535|14.424691358024692|
| stddev|29.06220650087229|1.1712419258042726|  8.56772896516207|
|    min|                0|               1.0|                 0|
|    max|               99|               5.0|                29|
+-------+-----------------+------------------+------------------+



In [48]:
test_data.describe().show()

+-------+-----------------+------------------+-----------------+
|summary|          movieId|            rating|           userId|
+-------+-----------------+------------------+-----------------+
|  count|              286|               286|              286|
|   mean|47.89510489510489|1.8461538461538463|14.20979020979021|
| stddev|28.39958321579849|1.2526894145121403|8.702371644358534|
|    min|                0|               1.0|                0|
|    max|               99|               5.0|               29|
+-------+-----------------+------------------+-----------------+



In [49]:
# Setting up Recommender system
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [50]:
# Build the recommendation model using ALS on the training data
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating')

In [51]:
model = als.fit(train_data)

Now let's see how the model performed!

In [53]:
predictions = model.transform(test_data)

In [55]:
predictions.show(4)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    29| 1.8932023|
|     85|   1.0|    13| 1.7420578|
|     85|   2.0|    20| 2.4327645|
|     85|   1.0|     5|  1.084132|
+-------+------+------+----------+
only showing top 4 rows



Evaluating the model by computing the RMSE on the test data


In [56]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='rmse')

In [57]:
rmse = evaluator.evaluate(predictions)

In [58]:
print("Root Mean Squared Error:",rmse)

Root Mean Squared Error: 1.0389889515220903


**The RMSE 1.03 described our error in terms of the stars rating column.**

## Recommendation for a particular user

In [59]:
single_user = test_data.select('userId','movieId').filter("userId == 11")

In [60]:
single_user.show()

+------+-------+
|userId|movieId|
+------+-------+
|    11|      0|
|    11|     10|
|    11|     18|
|    11|     23|
|    11|     25|
|    11|     32|
|    11|     35|
|    11|     45|
|    11|     67|
|    11|     70|
|    11|     76|
|    11|     79|
|    11|     89|
|    11|     90|
|    11|     99|
+------+-------+



In [61]:
recommendations = model.transform(single_user)

In [62]:
recommendations.orderBy('prediction', ascending=False).show()

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|    11|     32|  4.335572|
|    11|     18|  4.318816|
|    11|     23|  2.937516|
|    11|     35| 2.8498726|
|    11|     76| 2.2594368|
|    11|     79| 2.2459607|
|    11|     10| 2.1991081|
|    11|     90| 1.7375188|
|    11|     89| 1.5707614|
|    11|     67| 1.2681143|
|    11|     99|  1.116628|
|    11|      0| 1.0991439|
|    11|     70|  1.006738|
|    11|     25|0.97288835|
|    11|     45|0.82458675|
+------+-------+----------+



In [ ]:
#Closing the spark session
spark.stop()